# defining a database

Here we aim at defining a database of faces which are looking either at the left, to the centre or to the right of the screen. This will be used for further supervised learning.

For a full definition, see ``2018-03-20_LeCheapEyeTracker_build_database``


In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
np.set_printoptions(precision=2, suppress=True)
# define plots to be inserted interactively
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

## a minimalistic stimulation within the notebook


Instead of using an external program to show stimuli, we will use a minimalist solution within the notebook. This will consist of chains of characters that we really display into segments:

In [2]:
targets = {}
s_target = '🔴'
s_distra = '🔵'
s_target_bis = '🔵'
s_distra = '🌫'
s_blink = '☠️'

N_dis = 36 # <<<<<<<<<<<< change to adapt to your notebook's width
N_margin = 2

targets['left'] = N_margin * s_distra + s_target + 2 * N_dis * s_distra + N_margin * s_distra

targets['blink'] = N_margin * s_distra + N_dis * s_distra + s_blink + N_dis * s_distra + N_margin * s_distra

targets['center'] = N_margin * s_distra + N_dis * s_distra + s_target + N_dis * s_distra + N_margin * s_distra

targets['right'] = N_margin * s_distra + 2 * N_dis * s_distra + s_target + N_margin * s_distra

print ('Empty line')
null = N_margin * s_distra +  (2 * N_dis + 1) * s_distra + N_margin * s_distra
print (null)
print ('Fixation dot')
center = N_margin * s_distra + N_dis * s_distra + s_target_bis + N_dis * s_distra + N_margin * s_distra
print (center)

for label in targets.keys():
    print('label=', label)
    print (targets[label])

Empty line
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
Fixation dot
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🔵🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
label= left
🌫🌫🔴🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
label= blink
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫☠️🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
label= center
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🔴🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
label= right
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🔴🌫🌫


## wrapping things up

All the protocol in one script:

In [3]:
datapath = 'dataset'
import imageio
import os
from LeCheapEyeTracker.EyeTrackerServer import Server
from IPython.display import clear_output

N_category = len(targets.keys())
def get_stack(datapath=datapath, subject_name='anonymous', N_frame=64, startup_time=1., interframe_time=1., waiting_time=.2):
    try:
        import time
        time.sleep(startup_time)
        et = Server()
        print ('Time', et.clock())
        
        bootstraped, max_trials, no_trials = False, 100, 0
        while not bootstraped:
            try:
                img0 = et.cam.grab()
                img_face, res, t0 = et.process_frame(img0, et.clock())
                bootstraped = True
            except:
                if no_trials>max_trials:
                    raise('Failed!')
                else:
                    pass
            no_trials += 1

        H, W, three = img_face.shape
        for i in range(N_frame):            
            clear_output()
            try:
                # presentation of stimulus
                i_choice = np.random.randint(N_category)
                label = list(targets.keys())[i_choice]
                print(null)
                print(null)
                print(targets[label])
                print(null)
                print(null)

                time.sleep(interframe_time)
                frame = et.cam.grab()
                img_face, res, t0 = et.process_frame(frame, et.clock())

                clear_output()
                print(null)
                print(null)
                print(center)
                print(null)
                print(null)

                time.sleep(waiting_time)

                filename = os.path.join(datapath, label, subject_name + '_' + str(i) + '.png')
                imageio.imwrite(filename, img_face[:, :, ::-1])
            except Exception as e:
                print('Failed for trial ', i, 'with message', e)
    finally:
        et.close()

get_stack(subject_name='2018-09-04_Laurent')

🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🔵🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
Capture released


In [4]:
!ls -lR {datapath}

total 0
drwxr-xr-x   94 lolo  staff  3196 Sep  4 09:09 blink
drwxr-xr-x  288 lolo  staff  9792 Sep  4 09:09 center
drwxr-xr-x  260 lolo  staff  8840 Sep  4 09:09 left
drwxr-xr-x  216 lolo  staff  7344 Sep  4 09:09 right

dataset/blink:
total 10224
-rw-r--r--  1 lolo  staff  55224 May 24 15:57 2018-03-22_Laurent_blinks_17.png
-rw-r--r--  1 lolo  staff  59297 May 24 15:57 2018-03-22_Laurent_blinks_18.png
-rw-r--r--  1 lolo  staff  60292 May 24 15:57 2018-03-22_Laurent_blinks_19.png
-rw-r--r--  1 lolo  staff  54214 May 24 15:57 2018-03-22_Laurent_blinks_22.png
-rw-r--r--  1 lolo  staff  58117 May 24 15:57 2018-03-22_Laurent_blinks_26.png
-rw-r--r--  1 lolo  staff  58079 May 24 15:57 2018-03-22_Laurent_blinks_3.png
-rw-r--r--  1 lolo  staff  59208 May 24 15:57 2018-03-22_Laurent_blinks_37.png
-rw-r--r--  1 lolo  staff  59125 May 24 15:57 2018-03-22_Laurent_blinks_40.png
-rw-r--r--  1 lolo  staff  58707 May 24 15:57 2018-03-22_Laurent_blinks_42.png
-rw-r--r--  1 lolo  staff  54497 May 24 15